## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-30-18-43-00 +0000,wsj,"U.S., Ukrainian Negotiators Meet in Florida to...",https://www.wsj.com/politics/policy/u-s-ukrain...
1,2025-11-30-18-42-01 +0000,nypost,Refugee group was warned that DC shooting susp...,https://nypost.com/2025/11/30/us-news/refugee-...
2,2025-11-30-18-41-10 +0000,nyt,Why Is the U.S. Threatening Venezuela?,https://www.nytimes.com/video/world/americas/1...
3,2025-11-30-18-35-00 +0000,wsj,Opinion | How to Fix College Football,https://www.wsj.com/opinion/how-to-fix-college...
4,2025-11-30-18-30-30 +0000,nyt,Hong Kong’s Migrant Domestic Workers Mourn The...,https://www.nytimes.com/2025/11/30/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
196,trump,17
105,up,12
35,deadly,8
12,shooting,8
195,pardon,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
27,2025-11-30-17-12-00 +0000,wsj,Netanyahu Asks for a Pardon After Trump Says H...,https://www.wsj.com/world/middle-east/netanyah...,45
143,2025-11-29-22-53-36 +0000,bbc,Trump to pardon ex-Honduras president convicte...,https://www.bbc.com/news/articles/c1kpm0rvxepo...,43
136,2025-11-30-00-16-05 +0000,nyt,The Ex-President Whom Trump Plans to Pardon Fl...,https://www.nytimes.com/2025/11/29/nyregion/ho...,43
168,2025-11-29-20-06-00 +0000,wsj,Trump Declares That Airspace Around Venezuela ...,https://www.wsj.com/politics/national-security...,42
146,2025-11-29-22-09-35 +0000,nyt,In Announcing Pardon of Drug Trafficker While ...,https://www.nytimes.com/2025/11/29/us/politics...,41


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
27,45,2025-11-30-17-12-00 +0000,wsj,Netanyahu Asks for a Pardon After Trump Says H...,https://www.wsj.com/world/middle-east/netanyah...
85,38,2025-11-30-10-00-31 +0000,nypost,First images from inside fire-ravaged Hong Kon...,https://nypost.com/2025/11/30/world-news/hong-...
46,37,2025-11-30-15-37-28 +0000,nypost,DC National Guard terror suspect ‘radicalized’...,https://nypost.com/2025/11/30/us-news/dc-natio...
121,34,2025-11-30-03-50-26 +0000,nypost,"New York, New Jersey could get up to 5 inches ...",https://nypost.com/2025/11/29/us-news/ny-nj-co...
53,31,2025-11-30-13-49-43 +0000,cbc,Indonesian residents hunt for food and water a...,https://www.cbc.ca/news/world/death-tolls-rise...
74,26,2025-11-30-11-15-39 +0000,nypost,Netanyahu submits request to Israeli president...,https://nypost.com/2025/11/30/world-news/netan...
113,26,2025-11-30-05-08-56 +0000,nypost,Mass shooting at little girl’s birthday party ...,https://nypost.com/2025/11/30/us-news/californ...
23,25,2025-11-30-17-26-41 +0000,nyt,Lebanon’s Christians Look to Pope Leo’s Visit ...,https://www.nytimes.com/2025/11/30/world/middl...
90,22,2025-11-30-10-00-00 +0000,wapo,Why you may not want lower prices as much as y...,https://www.washingtonpost.com/business/2025/1...
5,20,2025-11-30-18-26-58 +0000,nypost,Manhattan US Attorney Jay Clayton to target cr...,https://nypost.com/2025/11/30/us-news/manhatta...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
